In [ ]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    """Load configuration from YAML file."""
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

# === Extract Balance, Equity, and Floating Loss === #
balance = account_info.balance
equity = account_info.equity
floating_loss = account_info.profit

# === Display Results === #
print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Fetch and Process Running Trades === #
fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
df = fetcher.get_sheet_as_dataframe("Forex")
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
else:
    # Filter and rename columns
    df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
    df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
    df_positions.rename(columns={
        "symbol": "Symbol",
        "type": "Action",
        "volume": "Lot",
        "price_open": "Price",
        "price_current": "Price Current",
        "sl": "SL",
        "tp": "TP",
        "profit": "Profit"
    }, inplace=True)

    # Aggregate by Symbol and Action
    def aggregate_group(group):
        symbol = group["Symbol"].iloc[0]
        action = group["Action"].iloc[0]
        lot = group["Lot"].sum()
        warnings = []
        print(symbol)

        # === Lot mismatch check === #
        df_lot_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[5]].sum()
        if not pd.isna(df_lot_value) and round(df_lot_value, 2) > round(lot, 2):
            warnings.append(f"Lot Mismatch: Expected {round(df_lot_value, 2)}, Found {round(lot, 2)}")

        # === SL/TP mismatch check === #
        df_sl_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[7]].sum()
        sl_mode = group["SL"].mode().iloc[0] if not group["SL"].mode().empty else np.nan
        if group["SL"].nunique() > 1 or (df_sl_value != 0 and round(df_sl_value, 2) != round(sl_mode, 2)):
            warnings.append("SL Mismatch")

        df_tp_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[8]].sum()
        tp_mode = group["TP"].mode().iloc[0] if not group["TP"].mode().empty else np.nan
        if group["TP"].nunique() > 1 or round(df_tp_value, 2) != round(tp_mode, 2):
            warnings.append("TP Mismatch")

        # Combine warnings
        warning_message = "; ".join(warnings) if warnings else np.nan

        return pd.Series({
            "Lot": lot,
            "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
            "Price Current": group["Price Current"].mean(),
            "SL": group["SL"].mode().iloc[0] if not group["SL"].empty else np.nan,
            "TP": group["TP"].mode().iloc[0] if not group["TP"].empty else np.nan,
            "Profit": group["Profit"].sum(),
            "Warning": warning_message
        })

    df_positions = (
        df_positions.groupby(["Symbol", "Action"], as_index=False)
        .apply(lambda group: aggregate_group(group))
        .reset_index(drop=True)
    )

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Calculate Coefficients === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"])) / (df_positions["Lot"] * 100)

# === Update Coefficients in Config (Exclude NaN) === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}

for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    
    # Only update if coefficient is not NaN
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        
        # Add or update the coefficient
        config["symbol_coefficients"][symbol] = avg_coeff

# === Save Updated Configuration === #
with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

# === Display Changed Coefficients === #
print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# Creating new rows for Balance, Equity, and Floating Loss
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

# Inserting the new rows at the top of the DataFrame
df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Uploade Updated Dataframe === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Display Final Merged DataFrame === #
print("Final Merged DataFrame:")
df_final

=== Account Information ===
Balance: 4286.29 USD
Equity: 4471.34 USD
Floating Loss: -1075.81 USD


In [ ]:
df.loc[(df["Coeff"] == "ATOMUSD")]

,Coeff,60,Unnamed: 2,Unnamed: 3,Unnamed: 4,3.28,2.96,0.32,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
24,ATOMUSD,Crypto,Buy,5.349333,False,0.055905,0.06,0,5.571172,-0.031655,...,0.040158,5.18,4.175318,8.404778,2.49756,Sideways,1093.914055,-100,-0.031655,11828.3765
